### **Avance 2 - Proyecto Final**

In [1]:
## Conectarse a la base de datos

from src.db.database import DBConnection
db1 = DBConnection()

In [2]:
# Ejecutar una query utilizando el método execute_query
# y guardar el resultado en un DataFrame de pandas

query = """
select FirstName as Nombre, LastName as Apellido, Gender as Genero, HireDate as FechaContratacion,
CityName as Ciudad, CountryName as Pais from employees e join cities c on e.cityid = c.cityid
join countries co on c.countryID= co.countryID LIMIT 5;
"""
df = db1.execute_query(query)
df

,Nombre,Apellido,Genero,FechaContratacion,Ciudad,Pais
0,Nicole,Fuller,F,2011-06-20 07:15:37,New Orleans,United States
1,Christine,Palmer,F,2011-04-27 04:07:57,Fremont,United States
2,Pablo,Cline,M,2012-03-30 18:55:23,Rochester,United States
3,Darnell,Nielsen,M,2014-03-06 06:55:03,Lubbock,United States
4,Desiree,Stuart,F,2014-11-16 22:59:55,Anaheim,United States


### **Patrones de diseño**

**Patron Singleton**

In [3]:
## Verificar Patron Singleton

db1 = DBConnection()
db2 = DBConnection()

print("Es la misma instancia?", db1 is db2)
print("Es la misma base de datos?", db1.engine.url == db2.engine.url)

Es la misma instancia? True
Es la misma base de datos? True


In [4]:
# Esto no creará una nueva conexión, sino que reutilizará la existente

db = DBConnection()

**Patron Factory**

In [5]:
## Implementación del patrón Factory

## SalesSummary
## Esta clase usa el patrón Factory y permite crear objetos de tipo SalesSummary a partir de un DataFrame de pandas. Lo que permite realizar una sola vez una query
## compleja y luego crear múltiples instancias de SalesSummary sin necesidad de repetir la consulta a la base de datos.

from src.design_patterns.factory import SalesSummary

query_sales = """
select SalesID, s.ProductID, ProductName, Quantity, TotalPrice, c.CustomerID, coalesce(concat( c.LastName, ", ", c.FirstName, " ", c.MiddleInitial, "."), "Sin nombre") as CustomerName,
e.EmployeeID, concat(e.LastName, ", ", e.FirstName, " ", e.MiddleInitial, ".") as EmployeeName
from sales s join products p on s.productid = p.ProductID
join customers c on s.CustomerID = c.CustomerID 
join employees e on s.SalesPersonID = e.EmployeeID
order by c.CustomerID;
"""
df_sales = db.execute_query(query_sales)
sales = [SalesSummary.from_series(row) for _, row in df_sales.iterrows()]
sales[0].__dict__

{'sale_id': 2494538,
 'product_id': 405,
 'product_name': 'Pastry - Raisin Muffin - Mini',
 'quantity': 1,
 'customer_id': 1,
 'customer_name': 'Frye, Stefanie Y.',
 'employee_id': 2,
 'employee_name': 'Palmer, Christine W.',
 'total_price': Decimal('2.00')}

In [6]:
## Implementación del patrón Factory

## CustomerLocationInfo
# Esta clase usa el patrón Factory y permite crear objetos de tipo CustomerLocationInfo a partir de un DataFrame de pandas. 
# Lo que permite realizar una sola vez una query

from src.design_patterns.factory import CustomerLocationInfo

## CustomerLocationInfo
query_customer_location = """
select CustomerID, FirstName, coalesce(MiddleInitial, "") as MiddleInitial, LastName, Address, CityName, CountryName
from customers cu join cities ci on cu.cityID = ci.CityID
join countries co on ci.CountryID = co.CountryID;
"""
df_customer_location = db.execute_query(query_customer_location)
customer_locations = [CustomerLocationInfo.from_series(row) for _, row in df_customer_location.iterrows()]
customer_locations[0].__dict__


{'customer_id': 103,
 'first_name': 'Jana',
 'middle_initial': 'Q',
 'last_name': 'Huang',
 'address': '841 Hague Road',
 'city_name': 'Dayton',
 'country_name': 'United States'}

In [7]:
## Implementación del patrón Strategy
# reutilizamos la query obtenida anteriormente para generar reportes de ventas totales por empleado.
# esta clase usa el patrón strategy y la estrategia es "TotalSalesByEmployee", que calcula las ventas totales por empleado.

from src.design_patterns.strategy import TotalSalesByEmployee

report = TotalSalesByEmployee()
report.generate_report(df_sales,"EmployeeName")

,IDVendedor,Nombre Apellido Vendedor,TotalVentas
17,18,"Bartlett, Warren C.",512154.00
20,21,"Brewer, Devon D.",618744.00
13,14,"Buckley, Wendi G.",406812.00
11,12,"Chen, Lindsay M.",338664.00
2,3,"Cline, Pablo Y.",83925.00
5,6,"Collins, Holly E.",171600.00
6,7,"Cook, Chadwick P.",195111.00
10,11,"Dickson, Sonya E.",308726.00
7,8,"Dyer, Julie E.",241680.00
14,15,"Finley, Kari D.",416940.00


In [8]:
## Implementación del patrón Strategy
# tambien reutilizamos la query obtenida anteriormente para generar reportes de ventas promedio por empleado.
# esta clase usa el patrón strategy y la estrategia es "AverageSalesByEmployee", que calcula las ventas promedio por empleado.

from src.design_patterns.strategy import AverageSalesByEmployee

report = AverageSalesByEmployee()
report.generate_report(df_sales,"EmployeeName")

,IDVendedor,Nombre Apellido Vendedor,Promedio de ventas
17,18,"Bartlett, Warren C.",232.269388
20,21,"Brewer, Devon D.",275.241993
13,14,"Buckley, Wendi G.",181.450491
11,12,"Chen, Lindsay M.",156.210332
2,3,"Cline, Pablo Y.",39.662098
5,6,"Collins, Holly E.",78.499543
6,7,"Cook, Chadwick P.",91.903439
10,11,"Dickson, Sonya E.",143.460037
7,8,"Dyer, Julie E.",106.326441
14,15,"Finley, Kari D.",197.414773


In [9]:
from src.design_patterns.strategy import ProductSalesByEmployee
report = ProductSalesByEmployee()
report.generate_report(df_sales, "ProductID", False)

,IDVendedor,Nombre Apellido Vendedor,Cantidad de productos vendidos
7,8,"Dyer, Julie E.",2273
1,2,"Palmer, Christine W.",2252
20,21,"Brewer, Devon D.",2248
13,14,"Buckley, Wendi G.",2242
15,16,"Walton, Chadwick U.",2205
17,18,"Bartlett, Warren C.",2205
22,23,"Flowers, Janet K.",2190
5,6,"Collins, Holly E.",2186
9,10,"Vang, Jean P.",2179
12,13,"Marks, Katina Y.",2177


In [10]:
## Implementación del patron Builder
# En esta celda se utiliza el patrón de diseño Builder para generar y combinar múltiples reportes de ventas a partir de un mismo DataFrame.
# ReportBuilder permite agregar diferentes estrategias de reporte (por ejemplo, ventas totales, promedio y por producto) 
# y construir todos los informes de manera flexible y escalable.
# Finalmente, se obtiene un reporte combinado ("CombinedReport") que integra los resultados de todas las estrategias aplicadas, 
# facilitando el análisis comparativo de la información.
# tambien puede generar reportes individuales como "TotalSalesByEmployee", "AverageSalesByEmployee" y "ProductSalesByEmployee".

from src.design_patterns.builder import ReportBuilder

builder = ReportBuilder()

reports = (
    builder.set_dataframe(df_sales)
    .set_combined_sorting("EmployeeName", True)
    .add_report(TotalSalesByEmployee())
    .add_report(AverageSalesByEmployee())
    .add_report(ProductSalesByEmployee())
    .build_all()
)

# Descomentar las siguientes líneas para ver los reportes individuales
# reports["ProductSalesByEmployee"]
# reports["TotalSalesByEmployee"]
# reports["AverageSalesByEmployee"]

# Reporte combinado que integra todos los reportes individuales
reports["CombinedReport"] 

,IDVendedor,Nombre Apellido Vendedor,TotalVentas,Promedio de ventas,Cantidad de productos vendidos
0,18,"Bartlett, Warren C.",512154.00,232.269388,2205
1,21,"Brewer, Devon D.",618744.00,275.241993,2248
2,14,"Buckley, Wendi G.",406812.00,181.450491,2242
3,12,"Chen, Lindsay M.",338664.00,156.210332,2168
4,3,"Cline, Pablo Y.",83925.00,39.662098,2116
5,6,"Collins, Holly E.",171600.00,78.499543,2186
6,7,"Cook, Chadwick P.",195111.00,91.903439,2123
7,11,"Dickson, Sonya E.",308726.00,143.460037,2152
8,8,"Dyer, Julie E.",241680.00,106.326441,2273
9,15,"Finley, Kari D.",416940.00,197.414773,2112


### **Tests**

In [11]:
# Verificar que la clase DBConnection sigue el patrón Singleton
# Se ejecuta el test para verificar que la clase DBConnection sigue el patrón Singleton

!pytest tests/test_singleton_instance.py -v


============================= test session starts =============================
platform win32 -- Python 3.13.3, pytest-8.3.5, pluggy-1.6.0 -- C:\Users\veron\OneDrive\Documentos\curso Henry\Proyecto-Final\env-final\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\veron\OneDrive\Documentos\curso Henry\Proyecto-Final
configfile: pytest.ini
collecting ... collected 1 item

tests/test_singleton_instance.py::test_singleton_instance PASSED         [100%]

============================== 1 passed in 1.09s ==============================


In [12]:
# Testando la clase SalesSummary que implementa el patrón Factory

!pytest tests/test_factory.py -v

============================= test session starts =============================
platform win32 -- Python 3.13.3, pytest-8.3.5, pluggy-1.6.0 -- C:\Users\veron\OneDrive\Documentos\curso Henry\Proyecto-Final\env-final\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\veron\OneDrive\Documentos\curso Henry\Proyecto-Final
configfile: pytest.ini
collecting ... collected 3 items

tests/test_factory.py::test_sales_summary_creation PASSED                [ 33%]
tests/test_factory.py::test_customer_location_info_creation PASSED       [ 66%]
tests/test_factory.py::test_customer_location_info_missing_columns PASSED [100%]

============================== 3 passed in 0.42s ==============================


In [13]:
# Testeando las Clases que implementan el patrón Strategy:
# TotalSalesByEmployee, AverageSalesByEmployee y ProductSalesByEmployee

!pytest tests/test_strategy.py -v

============================= test session starts =============================
platform win32 -- Python 3.13.3, pytest-8.3.5, pluggy-1.6.0 -- C:\Users\veron\OneDrive\Documentos\curso Henry\Proyecto-Final\env-final\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\veron\OneDrive\Documentos\curso Henry\Proyecto-Final
configfile: pytest.ini
collecting ... collected 5 items

tests/test_strategy.py::test_product_sales_by_employee PASSED            [ 20%]
tests/test_strategy.py::test_product_by_employee_sorting_by_name PASSED  [ 40%]
tests/test_strategy.py::test_product_by_employee_sorting_by_total_price PASSED [ 60%]
tests/test_strategy.py::test_total_sales_by_employee PASSED              [ 80%]
tests/test_strategy.py::test_average_sales_by_employee PASSED            [100%]

============================== 5 passed in 0.40s ==============================


In [14]:
# Teasteando la clase ReportBuilder que implementa el patrón Builder
# Esta clase permite generar reportes de ventas utilizando diferentes estrategias y combinarlos en un reporte final.

!pytest tests/test_builder.py -v

============================= test session starts =============================
platform win32 -- Python 3.13.3, pytest-8.3.5, pluggy-1.6.0 -- C:\Users\veron\OneDrive\Documentos\curso Henry\Proyecto-Final\env-final\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\veron\OneDrive\Documentos\curso Henry\Proyecto-Final
configfile: pytest.ini
collecting ... collected 1 item

tests/test_builder.py::test_report_builder_add_report PASSED             [100%]

============================== 1 passed in 0.44s ==============================


## **Avance 3**

In [15]:
# Nos conectamos nuevamente a la base de datos, si existe una conexión abierta, se reutiliza la misma conexión.
db_av3 = DBConnection()

### Uso de CTE y de funciones de ventana

In [17]:
# Esta consulta obtiene el producto más vendido por categoría, utilizando una CTE (Common Table Expression) 
# para calcular el total vendido por producto y luego filtrando los resultados para obtener solo el producto 
# con mayor venta en cada categoría.

query = """
with ranked_products as (
select c.CategoryName, p.ProductName, sum(Quantity) as total_vendido,
dense_rank() over (
partition by c.CategoryName
order by sum(s.Quantity) desc 
) as ds
from products p join categories c on p.CategoryID = c.CategoryID
join sales s on s.ProductID = p.ProductID
group by c.CategoryName, p.ProductName
order by c.categoryName, ds)
select * from ranked_products
where ds<= 1
order by categoryname, ds;
"""

df_cte = db_av3.execute_query(query)
df_cte = df_cte.rename(columns={
    "CategoryName": "Categoría",
    "ProductName": "Producto",
    "total_vendido": "Total Vendido",
    "ds": "Ranking"
})
df_cte


,Categoría,Producto,Total Vendido,Ranking
0,Beverages,Garlic - Primerba; Paste,1937,1
1,Cereals,Brandy - Bar,1904,1
2,Confections,Sprouts - Alfalfa,1910,1
3,Dairy,Nantucket - Pomegranate Pear,1657,1
4,Grain,Creme De Banane - Marie,1611,1
5,Meat,Bread - Italian Roll With Herbs,1916,1
6,Poultry,Campari,1773,1
7,Produce,Guinea Fowl,1875,1
8,Seafood,Wine - Fume Blanc Fetzer,1713,1
9,Shell fish,Cake - Box Window 10x10x2.5,1882,1


In [18]:
## Esta consulta obtiene el total facturado por producto y categoría,
## así como el porcentaje que representa cada producto dentro de su categoría y del total general de ventas.

query = """
with ventas_por_categoria as (
    select
        c.CategoryID, c.CategoryName, p.ProductID, p.ProductName, SUM(s.TotalPrice) AS TotalFacturado
    from sales s
    join products p ON s.ProductID = p.ProductID
    join categories c ON p.CategoryID = c.CategoryID
    group by c.CategoryID, c.CategoryName, p.ProductID, p.ProductName
),
totales_categoria as (
    select
        CategoryID,
        SUM(TotalFacturado) AS TotalCategoria
    from ventas_por_categoria
    group by CategoryID
),
total_general_ventas as (
    select SUM(TotalFacturado) AS GranTotal from ventas_por_categoria
)
select
    v.CategoryID, v.CategoryName, v.ProductName, v.TotalFacturado, t.TotalCategoria, g.GranTotal,
    ROUND(100 * v.TotalFacturado / t.TotalCategoria, 2) AS PorcentajeEnCategoria,
    ROUND(100 * v.TotalFacturado / g.GranTotal, 2) AS PorcentajeEnTotal
from ventas_por_categoria v
join totales_categoria t ON v.CategoryID = t.CategoryID
cross join total_general_ventas g
order by v.CategoryName, PorcentajeEnCategoria DESC;
"""

df_cte = db_av3.execute_query(query)
df_cte = df_cte.rename(columns={
    "CategoryID": "ID Categoría",
    "CategoryName": "Categoría",
    "ProductName": "Producto",
    "TotalFacturado": "Total Facturado",
    "TotalCategoria": "Total en Categoría",
    "GranTotal": "Gran Total",
    "PorcentajeEnCategoria": "% en Categoría",
    "PorcentajeEnTotal": "% en Total"
})
df_cte

,ID Categoría,Categoría,Producto,Total Facturado,Total en Categoría,Gran Total,% en Categoría,% en Total
0,5,Beverages,Garlic - Primerba; Paste,23566.00,673480.00,7863619.00,3.50,0.30
1,5,Beverages,Halibut - Steaks,20606.00,673480.00,7863619.00,3.06,0.26
2,5,Beverages,Soup - Campbells; Cream Of,20252.00,673480.00,7863619.00,3.01,0.26
3,5,Beverages,Bread - Italian Corn Meal Poly,19882.00,673480.00,7863619.00,2.95,0.25
4,5,Beverages,Jolt Cola - Electric Blue,19774.00,673480.00,7863619.00,2.94,0.25
...,...,...,...,...,...,...,...,...
447,10,Snails,Longos - Chicken Wings,14899.00,644595.00,7863619.00,2.31,0.19
448,10,Snails,Mussels - Frozen,14838.00,644595.00,7863619.00,2.30,0.19
449,10,Snails,Sprouts - Baby Pea Tendrils,14326.00,644595.00,7863619.00,2.22,0.18
450,10,Snails,Pasta - Cheese / Spinach Bauletti,13081.00,644595.00,7863619.00,2.03,0.17


### Objetos SQL

In [19]:
## Stored Procedure
from sqlalchemy import text

db_sp = DBConnection()
engine = db_sp.engine

create_sp_sql = """
create procedure sp_porcentaje_producto_total(in v_cat_id int)
begin
	-- 1. ventas de todos los productos para calcular el total general
	with ventas_todas_categorias as (
		select
			c.CategoryID, c.CategoryName, p.ProductID, p.ProductName, SUM(s.TotalPrice) AS TotalFacturado
		from sales s
		join products p ON s.ProductID = p.ProductID
		join categories c ON p.CategoryID = c.CategoryID
		group by c.CategoryID, c.CategoryName, p.ProductID, p.ProductName
	),
    -- 2. filtramos las categorias que llega por parametro
    totales_por_categoria as(
		select * from ventas_todas_categorias
        where CategoryID = v_cat_id
    ),
    -- 3. calculamos los totales por la categoria
	totales_categoria as (
		select CategoryID, SUM(TotalFacturado) as TotalCategoria
        from totales_por_categoria
        group by categoryID
	),
    -- 4. calculamos el total general de ventas de todos los productos
	total_general_ventas as (
		select SUM(TotalFacturado) AS GranTotal from ventas_todas_categorias
	)
    -- usamos todo lo anterior para sacar informacion por producto
	select
		v.CategoryID, v.CategoryName, v.ProductName, v.TotalFacturado, t.TotalCategoria, g.GranTotal,
		ROUND(100 * v.TotalFacturado / t.TotalCategoria, 2) AS PorcentajeEnCategoria,
		ROUND(100 * v.TotalFacturado / g.GranTotal, 2) AS PorcentajeEnTotal
	from totales_por_categoria v
	join totales_categoria t ON v.CategoryID = t.CategoryID
	join total_general_ventas g
	order by v.CategoryName, PorcentajeEnCategoria DESC;
end
"""

with engine.begin() as conn:
    # Si ya existía, la borramos
    conn.execute(text("DROP PROCEDURE IF EXISTS sp_porcentaje_producto_total;"))
    # Ahora creamos la nueva versión
    conn.execute(text(create_sp_sql))

print("✅ Stored procedure creado satisfactoriamente.")


✅ Stored procedure creado satisfactoriamente.


In [20]:
## Llamamos al stored procedure
category_id = 1  # Cambia este valor según la categoría que quieras consultar

df_porcentajes = db_sp.call_procedure("sp_porcentaje_producto_total", [category_id])

df_porcentajes

,CategoryID,CategoryName,ProductName,TotalFacturado,TotalCategoria,GranTotal,PorcentajeEnCategoria,PorcentajeEnTotal
0,1,Confections,Soup - Campbells; Beef Barley,23726.00,998526.00,7863619.00,2.38,0.30
1,1,Confections,Sprouts - Alfalfa,22182.00,998526.00,7863619.00,2.22,0.28
2,1,Confections,Bread - Raisin Walnut Oval,21503.00,998526.00,7863619.00,2.15,0.27
3,1,Confections,Tomato - Tricolor Cherry,21453.00,998526.00,7863619.00,2.15,0.27
4,1,Confections,Soup - Campbells Tomato Ravioli,21333.00,998526.00,7863619.00,2.14,0.27
5,1,Confections,Chicken - Wieners,21277.00,998526.00,7863619.00,2.13,0.27
6,1,Confections,Phyllo Dough,20138.00,998526.00,7863619.00,2.02,0.26
7,1,Confections,Watercress,19841.00,998526.00,7863619.00,1.99,0.25
8,1,Confections,Sauce - Rosee,19806.00,998526.00,7863619.00,1.98,0.25
9,1,Confections,Tofu - Firm,19731.00,998526.00,7863619.00,1.98,0.25


In [21]:
## Creación de una vista
create_view_sql = """
create or replace view vw_resumen_ventas_producto AS
select
    p.ProductID,
    p.ProductName,
    c.CategoryName,
    ROUND(AVG(s.TotalPrice / NULLIF(s.Quantity, 0)), 2) AS PrecioUnitarioPromedio,
    SUM(s.Quantity) AS TotalUnidadesVendidas,
    SUM(s.TotalPrice) AS TotalFacturado,
    ROUND(SUM(s.TotalPrice) / NULLIF(SUM(s.Quantity), 0), 2) AS TicketPromedio
from sales s
join products p ON s.ProductID = p.ProductID
join categories c ON p.CategoryID = c.CategoryID
group by p.ProductID, p.ProductName, c.CategoryName
"""

## ejecutamos la creación de la vista
with engine.begin() as conn:
    # Creamos la vista, si ya existía, la reemplazamos
    conn.execute(text(create_view_sql))

print("✅ Vista creada satisfactoriamente.")

✅ Vista creada satisfactoriamente.


In [22]:
# Ejecutamos la vista para obtener los datos
df_productos = db_sp.query_view("vw_resumen_ventas_producto")
df_productos

,ProductID,ProductName,CategoryName,PrecioUnitarioPromedio,TotalUnidadesVendidas,TotalFacturado,TicketPromedio
0,15,Spoon - Soup; Plastic,Confections,11.98,1425,17186.00,12.06
1,27,Chocolate - Compound Coating,Confections,12.38,1478,17924.00,12.13
2,42,Bread - Raisin Walnut Oval,Confections,12.22,1641,21503.00,13.10
3,52,Turnip - White; Organic,Confections,12.12,1386,17271.00,12.46
4,62,Pears - Bosc,Confections,12.16,1308,15256.00,11.66
...,...,...,...,...,...,...,...
447,398,Wine - Wyndham Estate Bin 777,Produce,11.45,1730,19993.00,11.56
448,405,Pastry - Raisin Muffin - Mini,Produce,12.74,1384,18322.00,13.24
449,409,Skirt - 29 Foot,Produce,11.68,1792,21582.00,12.04
450,439,Beef - Chuck; Boneless,Produce,12.52,1497,18780.00,12.55


In [23]:
# Como podemos utilizar la vista:
# Productos con mayor ticket promedio
query_top_tickets = """
select * from vw_resumen_ventas_producto
order by TicketPromedio desc
limit 10;
"""

df_resultado = db.execute_query(query_top_tickets)
df_resultado

,ProductID,ProductName,CategoryName,PrecioUnitarioPromedio,TotalUnidadesVendidas,TotalFacturado,TicketPromedio
0,50,Pork - Kidney,Meat,13.41,1442,20367.00,14.12
1,378,Bread Foccacia Whole,Poultry,13.98,1469,20707.00,14.10
2,30,Cheese - Brie; Triple Creme,Seafood,13.43,1561,21540.00,13.80
3,254,Soup - Canadian Pea; Dry Mix,Confections,13.11,1216,16732.00,13.76
4,224,Flour - Pastry,Confections,13.07,1332,18317.00,13.75
5,236,Tomato - Tricolor Cherry,Confections,13.29,1561,21453.00,13.74
6,78,Wine - Magnotta - Belpaese,Dairy,13.11,1320,18105.00,13.72
7,308,Tea - Decaf Lipton,Seafood,13.33,1506,20563.00,13.65
8,393,Olive - Spread Tapenade,Confections,13.05,1382,18800.00,13.60
9,448,Gatorade - Xfactor Berry,Beverages,13.13,1161,15730.00,13.55


### Optimización y análisis de rendimiento: índices

In [26]:
# Crear índices para las tablas Products, Sales y Customers

db_sp.execute_ddl("CREATE INDEX idx_products_name ON products(ProductName);")

In [27]:
db_sp.execute_query("SHOW INDEX FROM sales;")

,Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Visible,Expression
0,sales,0,PRIMARY,1,SalesID,A,46709,None,None,,BTREE,,,YES,None
1,sales,1,SalesPersonID,1,SalesPersonID,A,21,None,None,YES,BTREE,,,YES,None
2,sales,1,CustomerID,1,CustomerID,A,39142,None,None,YES,BTREE,,,YES,None
3,sales,1,idx_sales_product,1,ProductID,A,450,None,None,YES,BTREE,,,YES,None


In [25]:
## Borrar índices si existen
db_sp.execute_ddl("DROP INDEX idx_products_name ON products")